In [ ]:
# Selenium: 동적 웹페이지 조작
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

# 데이터 처리
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib import parse
from tqdm import tqdm
import time
import re

## 2. Chrome WebDriver 초기화

# 00. 토양 데이터 Selenium 크롤링

농촌진흥청 흙토람(soil.rda.go.kr)에서 **전국 읍면동별 농작물 재배 적합도 데이터**를 수집합니다.

## 데이터 수집 개요

| 항목 | 내용 |
|------|------|
| **대상 사이트** | 농촌진흥청 흙토람 (soil.rda.go.kr) |
| **수집 데이터** | 64개 농작물 × 전국 읍면동 = **32만+ 파라미터** |
| **수집 방법** | Selenium + BeautifulSoup |
| **소요 시간** | 파라미터 수집 ~1.5시간, 데이터 변환 ~5.5시간 |
| **결과 파일** | 33개 CSV 파일로 분할 저장 |

## 크롤링 프로세스

```
1. Selenium으로 사이트 접속 및 드롭다운 조작
   └── 시도 → 시군구 → 읍면동 순차 선택

2. 파라미터 수집 (32만개)
   └── 각 읍면동별 작물 목록 추출

3. API 요청으로 데이터 수집
   └── requests.post()로 토양 적합도 데이터 요청

4. XML 파싱 및 CSV 저장
   └── BeautifulSoup으로 파싱 후 분할 저장
```

## 주의사항

⚠️ **이 노트북은 참고용입니다.** 실제 크롤링은 약 7시간 이상 소요됩니다.  
⚠️ 수집된 데이터는 이미 `data/processed/` 폴더에 저장되어 있습니다.

---

## 1. 라이브러리 임포트

크롤링에 필요한 라이브러리들입니다.

In [ ]:
# Chrome WebDriver 자동 설치 및 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

## 3. 흙토람 사이트 접속

In [ ]:
# 농촌진흥청 흙토람 - 작물별 토양환경 페이지
driver.get('http://soil.rda.go.kr/soil/chart/chart.jsp')

## 4. 전국 읍면동 파라미터 수집

시도 → 시군구 → 읍면동 순서로 드롭다운을 순회하며 모든 파라미터를 수집합니다.

**수집 결과:** 321,664개 파라미터 (64개 농작물 × 전국 읍면동)

### 4.1 읍면동 목록 출력 (테스트)

In [ ]:
from selenium.webdriver.support.select import Select
import time
for sido_option in select_sido.options:
    sido_value = sido_option.get_attribute("value")
    
    if not sido_value:
        continue
        
    select_sido.select_by_value(sido_value)
    sido_nm = sido_option.text

    time.sleep(2)
    select_gungu_elem = driver.find_element(
        By.CSS_SELECTOR,
        '#sgg_cd_mini'
    )
    select_gungu = Select(select_gungu_elem)
    for gungu_option in select_gungu.options:
        gungu_value = gungu_option.get_attribute("value")
        gungu_nm = gungu_option.text
        if not gungu_value:
            continue
        select_gungu.select_by_value(gungu_value)
        time.sleep(2)
        select_umd_elem = driver.find_element(
            By.CSS_SELECTOR,
            '#umd_cd_mini'
        )
        select_umd = Select(select_umd_elem)
        for umd_option in

select_sido_elem = driver.find_element(
    By.CSS_SELECTOR,
    '#sido_cd_mini',
)
select_sido = Select(select_sido_el select_umd.options:
            umd_value = umd_option.get_attribute("value")
            umd_nm = umd_option.text
            print(sido_nm, gungu_nm, umd_nm)
            if not umd_value:
                continue
            select_umd.select_by_value(umd_value)

### 4.2 전체 파라미터 수집 (약 1.5시간 소요)

각 읍면동의 작물별 요청 파라미터를 추출합니다.

### 4.3 파라미터 DataFrame 변환

In [ ]:
from selenium.webdriver.support.select import Select

parameter_list = []
select_sido_elem = driver.find_element(
    By.CSS_SELECTOR,
    '#sido_cd_mini'
)
select_sido = Select(select_sido_elem)
for sido_option in select_sido.options:
    sido_value = sido_option.get_attribute("value")
    
    if not sido_value:
        continue
        
    select_sido.select_by_value(sido_value)
    sido_nm = sido_option.text

    time.sleep(2)
    select_gungu_elem = driver.find_element(
        By.CSS_SELECTOR,
        '#sgg_cd_mini'
    )
    select_gungu = Select(select_gungu_elem)
    for gungu_option in select_gungu.options:
        gungu_value = gungu_option.get_attribute("value")
        gungu_nm = gungu_option.text
        if not gungu_value:
            continue
        select_gungu.select_by_value(gungu_value)
        time.sleep(2)
        select_umd_elem = driver.find_element(
            By.CSS_SELECTOR,
            '#umd_cd_mini'
        )
        select_umd = Select(select_umd_elem)
        for umd_option in select_umd.options:
            umd_value = umd_option.get_attribute("value")
            umd_nm = umd_option.text
            fulladdr = "{} {} {}".format(sido_nm, gungu_nm, umd_nm)
            if not umd_value:
                continue
            select_umd.select_by_value(umd_value)
            
            categori_elem_list = driver.find_elements(
                By.CSS_SELECTOR,
                '#div_tab_a_1 > ul > li > dl > dd'
            )
            
            for category_dd in categori_elem_list:
                anchor_list = category_dd.find_elements(By.CSS_SELECTOR, "a")
                for anchor in anchor_list:
                    parameter = {}
                    a_href_text = anchor.get_attribute("href")
                    sel_cbo = 4
                    script_text = a_href_text.replace("javascript:go_Crop(", "").replace(");", "").replace("'", "").split(",")
                    sel_subcbo = script_text[0]
                    sel_cropcbo = script_text[1]
                    cropnm = script_text[2]
                    umd_cd = ''
                    flag = 2
                    sido_cd = sido_value
                    sgg_cd = gungu_value
                    umd_cd = umd_value
                    ri_cd = ''

                    parameter['sel_cbo']=sel_cbo
                    parameter['sel_subcbo']=sel_subcbo
                    parameter['sel_cropcbo']=sel_cropcbo
                    parameter['cropnm']=cropnm
                    parameter['umd_cd']=umd_cd
                    parameter['flag']=flag
                    parameter['fulladdr']=fulladdr
                    parameter['sido_cd']=sido_cd
                    parameter['sgg_cd']=sgg_cd
                    parameter['umd_cd']=umd_cd
                    parameter['ri_cd']=ri_cd
                    parameter['mode'] = 'chart.report'
                    parameter_list.append(parameter)
                    print(parameter)

In [45]:
param_df = pd.DataFrame(parameter_list)
param_df

,sel_cbo,sel_subcbo,sel_cropcbo,cropnm,umd_cd,flag,fulladdr,sido_cd,sgg_cd,ri_cd,mode
0,4,2,CR005,%EC%82%AC%EA%B3%BC,340,2,강원도 강릉시 강동면,42,150,,chart.report
1,4,2,CR006,%EB%B0%B0,340,2,강원도 강릉시 강동면,42,150,,chart.report
2,4,2,CR007,%ED%8F%AC%EB%8F%84,340,2,강원도 강릉시 강동면,42,150,,chart.report
3,4,2,CR008,%EA%B0%90%EA%B7%A4,340,2,강원도 강릉시 강동면,42,150,,chart.report
4,4,2,CR009,%EC%B0%B8%EB%8B%A4%EB%9E%98,340,2,강원도 강릉시 강동면,42,150,,chart.report
...,...,...,...,...,...,...,...,...,...,...,...
321659,4,8,CR063,%EB%A7%A5%EB%AC%B8%EB%8F%99,107,2,충청북도 충주시 호암동,43,130,,chart.report
321660,4,6,CR059,%EA%B3%B0%EC%B7%A8,107,2,충청북도 충주시 호암동,43,130,,chart.report
321661,4,6,CR060,%EC%B0%B8%EC%B7%A8,107,2,충청북도 충주시 호암동,43,130,,chart.report
321662,4,6,CR061,%EA%B3%A4%EB%8B%AC%EB%B9%84,107,2,충청북도 충주시 호암동,43,130,,chart.report


### 4.4 CSV 저장 및 로드

대용량 데이터이므로 CSV로 저장하여 재사용합니다.

In [16]:
param_df.to_csv('fruit_parameter.csv', encoding = 'cp949')

In [46]:
param_df = pd.read_csv("fruit_parameter.csv", encoding = 'cp949')
param_df = param_df.iloc[:,1:] # Unnamed 제거
param_df['ri_cd'] = '' # Nan값으로 되어있는 ri_cd 공백으로 대체
param_df

,sel_cbo,sel_subcbo,sel_cropcbo,cropnm,umd_cd,flag,fulladdr,sido_cd,sgg_cd,ri_cd,mode
0,4,2,CR005,%EC%82%AC%EA%B3%BC,340,2,강원도 강릉시 강동면,42,150,,chart.report
1,4,2,CR006,%EB%B0%B0,340,2,강원도 강릉시 강동면,42,150,,chart.report
2,4,2,CR007,%ED%8F%AC%EB%8F%84,340,2,강원도 강릉시 강동면,42,150,,chart.report
3,4,2,CR008,%EA%B0%90%EA%B7%A4,340,2,강원도 강릉시 강동면,42,150,,chart.report
4,4,2,CR009,%EC%B0%B8%EB%8B%A4%EB%9E%98,340,2,강원도 강릉시 강동면,42,150,,chart.report
...,...,...,...,...,...,...,...,...,...,...,...
321659,4,8,CR063,%EB%A7%A5%EB%AC%B8%EB%8F%99,107,2,충청북도 충주시 호암동,43,130,,chart.report
321660,4,6,CR059,%EA%B3%B0%EC%B7%A8,107,2,충청북도 충주시 호암동,43,130,,chart.report
321661,4,6,CR060,%EC%B0%B8%EC%B7%A8,107,2,충청북도 충주시 호암동,43,130,,chart.report
321662,4,6,CR061,%EA%B3%A4%EB%8B%AC%EB%B9%84,107,2,충청북도 충주시 호암동,43,130,,chart.report


In [47]:
parameter_list = param_df.to_dict('records')
parameter_list

[{'sel_cbo': 4,
  'sel_subcbo': 2,
  'sel_cropcbo': 'CR005',
  'cropnm': '%EC%82%AC%EA%B3%BC',
  'umd_cd': 340,
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': 42,
  'sgg_cd': 150,
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': 2,
  'sel_cropcbo': 'CR006',
  'cropnm': '%EB%B0%B0',
  'umd_cd': 340,
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': 42,
  'sgg_cd': 150,
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': 2,
  'sel_cropcbo': 'CR007',
  'cropnm': '%ED%8F%AC%EB%8F%84',
  'umd_cd': 340,
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': 42,
  'sgg_cd': 150,
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': 2,
  'sel_cropcbo': 'CR008',
  'cropnm': '%EA%B0%90%EA%B7%A4',
  'umd_cd': 340,
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': 42,
  'sgg_cd': 150,
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': 2,
  'sel_cropcbo': 'CR009',
  'cropnm': '%EC%B0%B8%EB%8B

In [17]:
parameter_list

[{'sel_cbo': 4,
  'sel_subcbo': '2',
  'sel_cropcbo': 'CR005',
  'cropnm': '%EC%82%AC%EA%B3%BC',
  'umd_cd': '340',
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': '42',
  'sgg_cd': '150',
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': '2',
  'sel_cropcbo': 'CR006',
  'cropnm': '%EB%B0%B0',
  'umd_cd': '340',
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': '42',
  'sgg_cd': '150',
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': '2',
  'sel_cropcbo': 'CR007',
  'cropnm': '%ED%8F%AC%EB%8F%84',
  'umd_cd': '340',
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': '42',
  'sgg_cd': '150',
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': '2',
  'sel_cropcbo': 'CR008',
  'cropnm': '%EA%B0%90%EA%B7%A4',
  'umd_cd': '340',
  'flag': 2,
  'fulladdr': '강원도 강릉시 강동면',
  'sido_cd': '42',
  'sgg_cd': '150',
  'ri_cd': '',
  'mode': 'chart.report'},
 {'sel_cbo': 4,
  'sel_subcbo': '2',
  'sel_cropcbo': 'CR

## 5. API 요청으로 토양 적합도 데이터 수집

### 5.1 코드 데이터 요청 (10,000개 배치)

전체 32만개를 한번에 처리하면 오류가 발생하므로, **10,000개 단위**로 분할 처리합니다.

```
parameter_list[0:10000]      → fruit_code_df0.csv
parameter_list[10001:20001]  → fruit_code_df1.csv
...
parameter_list[310001:321664] → fruit_code_df32.csv
```

### 5.2 API 요청 코드

각 배치마다 아래 코드를 실행하여 코드 데이터를 수집합니다.

```python
# 배치 범위 변경: [0:10000], [10001:20001], ...
for param in tqdm(parameter_list[10001:20001]):
```

## 6. XML 파싱으로 토양 데이터 추출

### 6.1 토양 적합도 데이터 파싱 (약 5.5시간 소요)

수집한 코드 데이터를 기반으로 실제 토양 적합도(최적지, 적지, 가능지 등)를 추출합니다.

In [56]:
start_time = time.time()

code_list = []
GET_CODE_URL = "http://soil.rda.go.kr/common/ajaxCall.do"

for param in tqdm(parameter_list[10001:20001]):
    try:
        response = requests.post(GET_CODE_URL, data=param)
        request_data = response.json()
        request_data['bjd_code'] = "{}{}{}00".format(param['sido_cd'], param['sgg_cd'], param['umd_cd'])
        code_list.append(request_data)
#         print(code_list)
    except Exception as e:
        print("ERROR :", param)
        print(e)

fruit_code_df1 = pd.DataFrame(code_list)
end_time = time.time()
print('--- 걸린시간: {} ---'.format(end_time - start_time))

 31%|█████████▍                    | 3137/10000 [30:19<1:03:55,  1.79it/s]

ERROR : {'sel_cbo': 4, 'sel_subcbo': 2, 'sel_cropcbo': 'CR005', 'cropnm': '%EC%82%AC%EA%B3%BC', 'umd_cd': 340, 'flag': 2, 'fulladdr': '강원도 고성군 수동면', 'sido_cd': 42, 'sgg_cd': 820, 'ri_cd': '', 'mode': 'chart.report'}
[Errno Expecting property name enclosed in double quotes] {dataCnt:0}: 1


 31%|██████████                      | 3138/10000 [30:19<53:01,  2.16it/s]

ERROR : {'sel_cbo': 4, 'sel_subcbo': 2, 'sel_cropcbo': 'CR006', 'cropnm': '%EB%B0%B0', 'umd_cd': 340, 'flag': 2, 'fulladdr': '강원도 고성군 수동면', 'sido_cd': 42, 'sgg_cd': 820, 'ri_cd': '', 'mode': 'chart.report'}
[Errno Expecting property name enclosed in double quotes] {dataCnt:0}: 1


 31%|██████████                      | 3139/10000 [30:20<52:40,  2.17it/s]

...

ERROR : {'sel_cbo': 4, 'sel_subcbo': 6, 'sel_cropcbo': 'CR061', 'cropnm': '%EA%B3%A4%EB%8B%AC%EB%B9%84', 'umd_cd': 315, 'flag': 2, 'fulladdr': '강원도 양구군 국토정중앙면', 'sido_cd': 42, 'sgg_cd': 800, 'ri_cd': '', 'mode': 'chart.report'}
[Errno Expecting property name enclosed in double quotes] {dataCnt:0}: 1


 88%|██████████████████████████▍   | 8832/10000 [1:24:50<07:51,  2.48it/s]

ERROR : {'sel_cbo': 4, 'sel_subcbo': 8, 'sel_cropcbo': 'CR056', 'cropnm': '%EC%9D%B8%EC%82%BC', 'umd_cd': 315, 'flag': 2, 'fulladdr': '강원도 양구군 국토정중앙면', 'sido_cd': 42, 'sgg_cd': 800, 'ri_cd': '', 'mode': 'chart.report'}
[Errno Expecting property name enclosed in double quotes] {dataCnt:0}: 1


100%|█████████████████████████████| 10000/10000 [1:36:02<00:00,  1.74it/s]

--- 걸린시간: 5762.787858963013 ---


In [57]:
fruit_code_df1

,dataCnt,param,titleImage,titleName,rdUrl,bjd_code
0,231,/rp [A.LAWD_CD] [42150340] [umd] [강원도 강릉시 강동면]...,,%EC%82%AC%EA%B3%BC,http://soil.rda.go.kr:80/RDServer/reports/soil...,4215034000
1,231,/rp [A.LAWD_CD] [42150340] [umd] [강원도 강릉시 강동면]...,,%EB%B0%B0,http://soil.rda.go.kr:80/RDServer/reports/soil...,4215034000
2,231,/rp [A.LAWD_CD] [42150340] [umd] [강원도 강릉시 강동면]...,,%ED%8F%AC%EB%8F%84,http://soil.rda.go.kr:80/RDServer/reports/soil...,4215034000
3,231,/rp [A.LAWD_CD] [42150340] [umd] [강원도 강릉시 강동면]...,,%EA%B0%90%EA%B7%A4,http://soil.rda.go.kr:80/RDServer/reports/soil...,4215034000
4,231,/rp [A.LAWD_CD] [42150340] [umd] [강원도 강릉시 강동면]...,,%EC%B0%B8%EB%8B%A4%EB%9E%98,http://soil.rda.go.kr:80/RDServer/reports/soil...,4215034000
...,...,...,...,...,...,...
9867,153,/rp [A.LAWD_CD] [42750355] [umd] [강원도 영월군 한반도면...,,%EC%B0%B8%EC%99%B8,http://soil.rda.go.kr:80/RDServer/reports/soil...,4275035500
9868,153,/rp [A.LAWD_CD] [42750355] [umd] [강원도 영월군 한반도면...,,%EB%94%B8%EA%B8%B0,http://soil.rda.go.kr:80/RDServer/reports/soil...,4275035500
9869,153,/rp [A.LAWD_CD] [42750355] [umd] [강원도 영월군 한반도면...,,%EC%98%A4%EC%9D%B4,http://soil.rda.go.kr:80/RDServer/reports/soil...,4275035500
9870,153,/rp [A.LAWD_CD] [42750355] [umd] [강원도 영월군 한반도면...,,%ED%86%A0%EB%A7%88%ED%86%A0,http://soil.rda.go.kr:80/RDServer/reports/soil...,4275035500


In [58]:
fruit_code_df1.to_csv("fruit_code_df1.csv", encoding="cp949")

### 6.2 결과 데이터 확인

수집된 토양 적합도 데이터입니다.

**컬럼 설명:**
- `법정동코드`: 10자리 행정구역 코드
- `작물이름`: 농작물명
- `최적지`: 최적 재배 가능 면적 (ha)
- `적지`: 재배 적합 면적 (ha)
- `가능지`: 재배 가능 면적 (ha)
- `저위생산지`: 저생산성 면적 (ha)
- `기타`: 기타 면적 (ha)
- `합계`: 총 면적 (ha)

### 6.3 결과 확인 및 저장

In [62]:
import re
from urllib import parse
start_time = time.time()

p = re.compile('\\(([^)]+)')
soil_data_list = []
DATA_REQUEST_URL = "http://www.naas.go.kr/report2/ReportingServer/service"
for request_data in tqdm(code_list):
    request_param = request_data['param']
    request_rdUrl = request_data['rdUrl']

    request_param = {
        'opcode' : '700',
        'mrd_path' : request_rdUrl,
        'mrd_param' : request_param + '/rchartopt [2] /rfn [http://www.naas.go.kr/report2/DataServer/rdagent.jsp] /rsn [soil] /rfonttype50u',
        'mrd_plain_param' : '',
        'mrd_data' : '',
        'runtime_param' : '',
        'mmlVersion' : '0',
        'protocol' : 'sync'
    }
    data_xml = requests.post(DATA_REQUEST_URL, request_param).content
    xml_soup = BeautifulSoup(data_xml, 'html.parser')
    datas = xml_soup.find_all("tl")[-15:-3]
    
    jakmul_nm_text = xml_soup.select_one('tl[bo="230"]').text
    jakmul_nm = parse.unquote(p.findall(jakmul_nm_text)[0])
    key_names = ["법정동코드", '작물이름', datas[0].text, datas[1].text, datas[2].text, datas[3].text, datas[4].text, datas[5].text]
    values = [request_data['bjd_code'], jakmul_nm, datas[6].text, datas[7].text, datas[8].text, datas[9].text, datas[10].text, datas[11].text]
    
    data = {key : value for key, value in zip(key_names, values)}
    soil_data_list.append(data)

product_df1 = pd.DataFrame(soil_data_list)

end_time = time.time()
print('--- 걸린시간: {} ---'.format(end_time - start_time))

100%|███████████████████████████████| 9872/9872 [5:01:46<00:00,  1.83s/it]

--- 걸린시간: 18106.53962993622 ---


In [63]:
product_df1

,법정동코드,작물이름,최적지,적지,가능지,저위생산지,기타,합계
0,4215034000,사과,467,500,609,"9,437",174,"11,187"
1,4215034000,배,836,467,"1,068","8,642",174,"11,187"
2,4215034000,포도,0,777,529,"9,705",174,"11,185"
3,4215034000,감귤,0,0,0,0,"11,187","11,187"
4,4215034000,참다래,595,65,480,"9,873",174,"11,187"
...,...,...,...,...,...,...,...,...
9867,4275035500,참외,934,251,306,"5,081",404,"6,976"
9868,4275035500,딸기,0,266,725,"5,580",404,"6,975"
9869,4275035500,오이,366,178,326,"5,701",404,"6,975"
9870,4275035500,토마토,157,850,383,"5,181",404,"6,975"


In [64]:
product_df1
product_df1.to_csv("product_df1.csv", encoding="cp949")

## 7. 다음 단계

수집된 33개의 CSV 파일을 병합하여 최종 데이터셋을 생성합니다.

```python
# 33개 파일 병합 예시
import glob

csv_files = glob.glob('product_df*.csv')
df_list = [pd.read_csv(f, encoding='cp949') for f in csv_files]
final_df = pd.concat(df_list, ignore_index=True)

# 면적당 비율 계산
final_df['면적당 최적지'] = final_df['최적지'] / final_df['합계']
final_df['면적당 적지'] = final_df['적지'] / final_df['합계']

final_df.to_csv('final_soil_ratio.csv', encoding='cp949', index=False)
```

---

## 크롤링 주요 어려움 및 해결

| 어려움 | 해결 방법 |
|--------|----------|
| 32만개 대용량 데이터 | 10,000개 단위 배치 처리 |
| 페이지 로딩 지연 | `time.sleep(2)` 대기 |
| 동적 드롭다운 | `Select` 클래스로 옵션 제어 |
| XML 파싱 복잡성 | BeautifulSoup + 정규표현식 |
| URL 인코딩 | `urllib.parse.unquote()` |